This notebook is written based on [this reference implementation](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb).

Other refs for model:
* https://stackoverflow.com/questions/65205582/how-can-i-add-a-bi-lstm-layer-on-top-of-bert-model
* https://discuss.pytorch.org/t/how-to-connect-hook-two-or-even-more-models-together/21033
* https://pytorch.org/tutorials/beginner/transformer_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Other refs for torchtext:
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-ii-f146c8b9a496
* http://anie.me/On-Torchtext/

# Imports and setup

In [1]:
import pandas as pd
import numpy as np
import os
import random
random.seed(1)
import re
import time

# Data processing.
import dataset # dataset.py
import torch
from torchtext.legacy import data 

# Model.
import models # models.py
import torch.nn as nn
import torch.optim as optim
from transformers import DistilBertModel, DistilBertTokenizer

# Training.
import training # training.py
from sklearn.model_selection import KFold
import utils # utils.py

# Visualization.
import matplotlib.pyplot as plt

# If you make a code change that doesn't get picked up by
# Jupyter notebook, try reloading like below:
# import imp
# imp.reload(training)

# Load a BERT tokenizer

In [2]:
WEIGHTS_NAME = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(WEIGHTS_NAME)

# Read the data

In [3]:
data_df = dataset.get_multiple_datasets([1,2,3], 'Creativity_Combined', shuffle=True)

In [4]:
# For prototype purposes:
# split into train, test sets. (Train set will be further split into 
# train+validation sets, via k-fold CV.)
train_df = data_df[:1000]
test_df = data_df[1000:] # roughly 190 test examples set aside

# write them to CSV files
train_df.to_csv('ktrain.csv', index=False, header=False)
test_df.to_csv('ktest.csv', index=False, header=False)

## Preprocessing and transform into torchtext Dataset format.

From what I understand, some preprocessing is done when data.Field() is applied.

In [5]:
INIT_TOKEN_IDX = tokenizer.cls_token_id
EOS_TOKEN_IDX = tokenizer.sep_token_id
PAD_TOKEN_IDX = tokenizer.pad_token_id
UNK_TOKEN_IDX = tokenizer.unk_token_id

# BERT input can be at most 512 words
MAX_INPUT_LENGTH = tokenizer.max_model_input_sizes[WEIGHTS_NAME]

# Apply tokenization and some preprocessing steps to the input sentence.
# Namely, this trims examples down to MAX_INPUT_LENGTH. (There is a -2 
# since the [CLS] and [SEP] tokens will be added)
def tokenize_and_cut(sentence):
  sentence = sentence.replace('/', '') # remove slashes
  tokens = tokenizer.tokenize(sentence) 
  tokens = tokens[:MAX_INPUT_LENGTH-2]
  return tokens

# text_fields defines preprocessing and handling of the text of an example.
text_fields = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = INIT_TOKEN_IDX, # add [CLS] token
                  eos_token = EOS_TOKEN_IDX, # add [SEP] token
                  pad_token = PAD_TOKEN_IDX,
                  unk_token = UNK_TOKEN_IDX)

# label_fields defines how to handle the label of an example.
# for regression, we do not need to build a vocabulary.
label_fields = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)
all_fields = [('text', text_fields), ('label', label_fields)] # must match order of cols in csv

train_dataset, test_dataset = data.TabularDataset.splits(
  path='', # path='' because the csvs are in the same directory
  train='ktrain.csv', test='ktest.csv', format='csv',
  fields=all_fields  
)

In [6]:
print(data_df.head(1))

                                                  text  label
126  My idea is for a punching bag that would measu...  3.975


In [7]:
# Transform train_dataset into an np array representation.
# This will be used for generating the K folds.
train_exs_arr = np.array(train_dataset.examples)

# Training pipeline begins here


## Define parameters

In [29]:
# Model parameters
OUTPUT_DIM = 1
DROPOUT = 0.2 # This is the distilbert default
HIDDEN_DIM = 256 
NUM_LAYERS = 1
BIDIRECTIONAL = False

# Training parameters
N_SPLITS = 5 # For cross validation
BATCH_SIZE = 8 # This is the distilbert default
N_EPOCHS = 3 # This is the distilbert default

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.MSELoss(size_average=False)
criterion = criterion.to(device)

## The cell where it actually trains!

In [30]:
import imp
imp.reload(models)

# The main training loop
def launch_experiment(train_data_df, n_splits=N_SPLITS):
  # valid_corrs = np.empty(n_splits)
  best_valid_loss = float('inf') 
  
  kf = KFold(n_splits=n_splits)
  fold = 0
  for train_index, valid_index in kf.split(train_data_df):
    print('training on fold {}'.format(fold))
    train_data = data.Dataset(train_exs_arr[train_index], all_fields)
    valid_data = data.Dataset(train_exs_arr[valid_index], all_fields)
    
    # Initialize a new model each fold.
    # https://ai.stackexchange.com/questions/18221/deep-learning-with-kfold-cross-validation-with-epochs
    # https://stats.stackexchange.com/questions/358380/neural-networks-epochs-with-10-fold-cross-validation-doing-something-wrong
    bert = DistilBertModel.from_pretrained(WEIGHTS_NAME)
    model = models.BERTRNN(bert,
                           OUTPUT_DIM,
                           HIDDEN_DIM,
                           NUM_LAYERS,
                           BIDIRECTIONAL,
                           DROPOUT)
    # optimizer = optim.Adam(model.parameters())
    optimizer = optim.Adam(model.parameters(),lr=5e-05,betas=(0.9, 0.999),eps=1e-08) # lr is not Adam default
    model = model.to(device)

    train_iterator, valid_iterator = training.get_iterators(train_data, valid_data, BATCH_SIZE, device)

    for epoch in range(N_EPOCHS):
      start_time = time.time()
      train_loss, train_corr = training.train(model, train_iterator, optimizer, criterion)
      valid_loss, valid_corr = training.evaluate(model, valid_iterator, criterion)
      # valid_corrs[fold] will end up holding the valid_corr from the last epoch for this fold
      # is this what we want?
      # valid_corrs[fold] = valid_corr
      end_time = time.time()
      epoch_mins, epoch_secs = utils.epoch_time(start_time, end_time)

      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        # Save the weights of the model with the best valid loss
        print('updating saved weights of best model')
        torch.save(model.state_dict(), "best_valid_loss.pt")

      print(f'Epoch: {epoch:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\t Train Loss: {train_loss:.3f} | Train Corr: {train_corr:.2f}')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Corr: {valid_corr:.2f}')
    
    fold += 1
    # return best_valid_loss # TODO REMOVE -- JUST TRAIN ONE FOLD
  
  return best_valid_loss # , valid_corrs

In [31]:
best_valid_loss = launch_experiment(train_exs_arr)
print('best validation loss: {}'.format(best_valid_loss))
# print('validation correlations for each fold: {}'.format(valid_corrs))
# print('average of validation correlations: {}'.format(np.mean(valid_corrs)))

training on fold 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


updating saved weights of best model
Epoch: 00 | Epoch Time: 0m 50s
	 Train Loss: 27.649 | Train Corr: -0.10
	 Val. Loss: 4.441 |  Val. Corr: 0.05
Epoch: 01 | Epoch Time: 0m 50s
	 Train Loss: 4.267 | Train Corr: 0.34
	 Val. Loss: 4.607 |  Val. Corr: 0.05
updating saved weights of best model
Epoch: 02 | Epoch Time: 0m 49s
	 Train Loss: 3.322 | Train Corr: 0.55
	 Val. Loss: 2.896 |  Val. Corr: 0.19
training on fold 1


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 49s
	 Train Loss: 21.031 | Train Corr: 0.04
	 Val. Loss: 4.972 |  Val. Corr: 0.00
Epoch: 01 | Epoch Time: 0m 50s
	 Train Loss: 3.784 | Train Corr: 0.44
	 Val. Loss: 5.893 |  Val. Corr: 0.16
Epoch: 02 | Epoch Time: 0m 49s
	 Train Loss: 3.228 | Train Corr: 0.54
	 Val. Loss: 3.271 |  Val. Corr: 0.30
training on fold 2


...

Epoch: 00 | Epoch Time: 0m 49s
	 Train Loss: 21.090 | Train Corr: -0.04
	 Val. Loss: 4.410 |  Val. Corr: -0.02
Epoch: 01 | Epoch Time: 0m 51s
	 Train Loss: 3.945 | Train Corr: 0.43
	 Val. Loss: 3.997 |  Val. Corr: 0.08
Epoch: 02 | Epoch Time: 0m 49s
	 Train Loss: 3.013 | Train Corr: 0.59
	 Val. Loss: 4.118 |  Val. Corr: 0.26
training on fold 3


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 0m 49s
	 Train Loss: 22.656 | Train Corr: -0.01
	 Val. Loss: 4.364 |  Val. Corr: -0.02
Epoch: 01 | Epoch Time: 0m 50s
	 Train Loss: 4.213 | Train Corr: 0.40
	 Val. Loss: 5.368 |  Val. Corr: 0.14
Epoch: 02 | Epoch Time: 0m 49s
	 Train Loss: 3.418 | Train Corr: 0.53
	 Val. Loss: 4.992 |  Val. Corr: 0.23
training on fold 4


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

# Test the trained model on held-out dataset.

In [32]:
# Get a test iterator
test_iterator = training.get_iterator(test_dataset, BATCH_SIZE, device)

In [34]:
# load the best model saved
bert = DistilBertModel.from_pretrained(WEIGHTS_NAME)
model = models.BERTRNN(bert, OUTPUT_DIM, HIDDEN_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT)
model.load_state_dict(torch.load("best_valid_loss.pt"))
model.to(device)
model.eval()
test_loss, test_corr = training.evaluate(model, test_iterator, criterion, debug=True)
print(test_loss)
print(test_corr)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


predictions: tensor([4.2996, 3.2968, 4.2469, 4.2325, 4.0204, 3.9585, 3.5086, 4.0845],
       device='cuda:0')
true labels: tensor([5.0500, 3.4750, 4.9250, 4.5250, 4.6750, 2.7250, 5.0250, 5.0500],
       device='cuda:0')
predictions: tensor([4.1842, 4.0674, 4.2900, 4.2038, 4.2717, 3.8140, 3.8429, 4.2308],
       device='cuda:0')
true labels: tensor([4.7000, 4.7000, 4.3750, 4.9500, 4.5000, 2.7500, 4.5000, 4.6000],
       device='cuda:0')
predictions: tensor([3.8111, 3.3462, 4.1930, 3.2977, 4.2316, 3.8145, 3.2369, 3.2244],
       device='cuda:0')
true labels: tensor([3.9250, 4.0750, 4.3000, 3.6250, 4.8500, 4.3500, 3.8500, 3.4750],
       device='cuda:0')
predictions: tensor([3.3156, 3.7073, 4.2682, 4.1414, 4.1914, 4.2587, 4.2474, 4.0988],
       device='cuda:0')
true labels: tensor([5.0500, 4.4500, 4.8000, 5.2000, 3.8000, 4.0250, 5.0250, 4.7000],
       device='cuda:0')
predictions: tensor([4.2166, 3.9875, 3.9715, 4.0421, 4.0757, 4.0330, 4.3168, 3.8786],
       device='cuda:0')
true label

# Misc other stuff

Link to the trainer class: https://huggingface.co/transformers/main_classes/trainer.html



Default training arguments: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments

Batch size per device: 8

Epoch: 3



This should be the model I used to generate my initial results: https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
"DistilBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks."